In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd.function import Function
import os
import pickle
import numpy as np
import pandas as pd
import math
from itertools import chain
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import itertools
import random
import tqdm
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fftshift

from einops import rearrange, repeat
from einops.layers.torch import Rearrange
from torch import nn, einsum
import math
import logging
from functools import partial
from collections import OrderedDict
from sklearn.metrics import classification_report
from torchsummary import summary

In [32]:
!nvidia-smi
torch.cuda.is_available()

Tue Aug 10 21:57:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:01:00.0 Off |                  N/A |
| 39%   35C    P8    35W / 260W |  10906MiB / 11016MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

True

In [23]:
!nvidia-smi -h

NVIDIA System Management Interface -- v460.91.03

NVSMI provides monitoring information for Tesla and select Quadro devices.
The data is presented in either a plain text or an XML format, via stdout or a file.
NVSMI also provides several management operations for changing the device state.

Note that the functionality of NVSMI is exposed through the NVML C-based
library. See the NVIDIA developer website for more information about NVML.
Python wrappers to NVML are also available.  The output of NVSMI is
not guaranteed to be backwards compatible; NVML and the bindings are backwards
compatible.

http://developer.nvidia.com/nvidia-management-library-nvml/
http://pypi.python.org/pypi/nvidia-ml-py/
Supported products:
- Full Support
    - All Tesla products, starting with the Kepler architecture
    - All Quadro products, starting with the Kepler architecture
    - All GRID products, starting with the Kepler architecture
    - GeForce Titan products, starting with the Kepler architecture
- L

In [25]:
!nvidia-smi -q


==============NVSMI LOG==============

Timestamp                                 : Tue Aug 10 20:29:31 2021
Driver Version                            : 460.91.03
CUDA Version                              : 11.2

Attached GPUs                             : 1
GPU 00000000:01:00.0
    Product Name                          : GeForce RTX 2080 Ti
    Product Brand                         : GeForce
    Display Mode                          : Enabled
    Display Active                        : Disabled
    Persistence Mode                      : Disabled
    MIG Mode
        Current                           : N/A
        Pending                           : N/A
    Accounting Mode                       : Disabled
    Accounting Mode Buffer Size           : 4000
    Driver Model
        Current                           : N/A
        Pending                           : N/A
    Serial Number                         : N/A
    GPU UUID                              : GPU-194e9f2f-a2f6-9b45-eb16-03

In [18]:
cd PreTdat

/home/is/zheng-ch/Transformer/PreTdat


In [3]:
dat1 = np.load('/project/hikaku_db/data/sleep_SHHS/stages_sig/C3_spec_30_np/spec_c3_5007_5793_30.npy')
dat2 = np.load('/project/hikaku_db/data/sleep_SHHS/stages_sig/C4_spec_30_np/spec_5007_5793_30.npy')
print(dat1.shape)
print(dat2.shape)

dat = np.concatenate((dat1.reshape(-1,1,32,30), dat2.reshape(-1,1,32,30)), axis=1)
print(dat.shape)

(786606, 32, 30)
(786606, 32, 30)
(786606, 2, 32, 30)


In [19]:
datgradient = dat[0:10000]
np.save('/home/is/zheng-ch/Transformer/PreTdat/testgradientdat', datgradient)

In [9]:
index = pd.read_csv("/project/hikaku_db/data/sleep_SHHS/stages_sig/ann_delrecords_5class.csv", header=None)
index = index.astype(int)
print(index[5007610:5794216].apply(pd.value_counts))
label = index[5007610:5794216].values.tolist()#list
print(len(label))

        0
2  317703
0  220808
4  118212
3   97754
1   32129
786606


In [20]:
listgradient = np.array(label[0:10000])
np.save('/home/is/zheng-ch/Transformer/PreTdat/testgradientdat_label', listgradient)

In [16]:
####start

In [ ]:
dat = np.load('testgradientdat.npy')
label = np.load('testgradientdat_lable.npy')

In [6]:
class Mydatasets(torch.utils.data.Dataset):
    def __init__(self, data,label ,transform = None):
        self.transform = transform

        self.data = data
        self.label = label

        self.datanum = len(data)

    def __len__(self):
        return self.datanum

    def __getitem__(self, idx):
        
        out_data = torch.tensor(self.data[idx]).float()
        out_label = torch.tensor(self.label[idx])
        if self.transform:
            out_data = self.transform(out_data)

        return out_data, out_label

In [26]:
print(dat.shape)
# label = index[0:100].values.tolist()#list
print(len(label))
fixdata = dat[:,:,0:16,:]
mean_p1 = np.mean(dat[:,:,16:20,:], axis = 2)
mean_p2 = np.mean(dat[:,:,20:24,:], axis = 2)
mean_p3 = np.mean(dat[:,:,24:28,:], axis = 2)
mean_p4 = np.mean(dat[:,:,28:32,:], axis = 2)
num_data = len(dat)
ch = 2
inputdat = np.concatenate((fixdata,mean_p1.reshape(num_data, ch, 1, 30),mean_p2.reshape(num_data, ch, 1, 30),mean_p3.reshape(num_data, ch, 1, 30),mean_p4.reshape(num_data, ch, 1, 30)),axis=2)
print(inputdat.shape)

train, test, train_label, test_label = train_test_split(inputdat, np.array(label),test_size = 0.1, stratify = label,random_state = 0)
print('train data:',len(train))
print('test data:',len(test))

train_data_set = Mydatasets(data=train,label=train_label)
test_data_set = Mydatasets(data=test,label=test_label)

train_dataloader = torch.utils.data.DataLoader(train_data_set, batch_size = 64, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_data_set, batch_size = 64, shuffle=False)

(786606, 2, 32, 30)
786606
(786606, 2, 20, 30)
train data: 707945
test data: 78661


In [8]:
##Transformer model, flatted the EEG channel, allows parallel computation the multi-band from multi-channel. 

class PreNorm(nn.Module): 
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn
    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.5):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

class Attention(nn.Module):
    def __init__(self, dim, heads, dim_head, dropout = 0.5):
        super().__init__()
        inner_dim = dim_head *  heads       ##32(4*8)   
        project_out = not (heads == 1 and dim_head == dim)

        self.heads = heads
        self.scale = dim_head ** -0.5

        self.attend = nn.Softmax(dim = -1)
        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)   ##better to dim to dim*3

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()

    def forward(self, x):
        b, n, _, h = *x.shape, self.heads
        qkv = self.to_qkv(x).chunk(3, dim = -1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = h), qkv)

        dots = einsum('b h i d, b h j d -> b h i j', q, k) * self.scale

        attn = self.attend(dots)

        out = einsum('b h i j, b h j d -> b h i d', attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)

class Transformer(nn.Module):           ##Register the blocks into whole network
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout = 0.5):
        super().__init__()
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm(dim, Attention(dim, heads = heads, dim_head = dim_head, dropout = dropout)),
                PreNorm(dim, FeedForward(dim, mlp_dim, dropout = dropout))
            ]))
    def forward(self, x):
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x
        return x

class ViT(nn.Module):
    def __init__(self, *, image_size, time_size, fre_size, num_classes, dim, depth, heads, mlp_dim, pool = 'cls', channels , dim_head, dropout = 0.5, emb_dropout = 0.):
        super().__init__()
        assert image_size == 30  ##Time dimensions must equal to 30s
        num_patches = 300       #30*5'
        patch_dim = time_size * fre_size    #1*4'
        assert pool in {'cls', 'mean'}, 'pool type must be either cls (cls token) or mean (mean pooling)'

        self.to_patch_embedding = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w c) (p1 p2)', p1 = fre_size, p2 = time_size),
            nn.Linear(patch_dim, dim),        ##4 to dim
        )

        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim)) ##Generate the pos value'
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))                ##Generate the class value'
        self.dropout = nn.Dropout(emb_dropout)

        self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)

        self.pool = pool
        self.to_latent = nn.Identity()

        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_classes)
        )

    def forward(self, img):
        x = self.to_patch_embedding(img)
        b, n, _ = x.shape

        cls_tokens = repeat(self.cls_token, '() n d -> b n d', b = b)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding[:, :(n + 1)]
        x = self.dropout(x)

        x = self.transformer(x)

        x = x.mean(dim = 1) if self.pool == 'mean' else x[:, 0]

        x = self.to_latent(x)
        return self.mlp_head(x)


In [11]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

Transmodel = ViT(
    image_size = 30, 
    time_size = 1, 
    fre_size = 4, 
    num_classes = 5, 
    dim = 32, 
    depth = 8, 
    heads = 4, 
    mlp_dim = 128, 
    channels = 2,
    dim_head = 16
).to(DEVICE)

#summary(Transmodel,(2286886, 20, 30))


criterion = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.AdamW(Transmodel.parameters(), lr=1e-3)

#Transmodel.load_state_dict(torch.load('PreTrained_t_Trans_8lay_50drop_mulchan_eegspec_stateTrain'))
Transmodel.load_state_dict(torch.load('PreTest_Trans_8lay_50drop_noW_eegspec_state6'))



cuda:0


<All keys matched successfully>

In [ ]:
optimizer = torch.optim.AdamW(Transmodel.parameters(), lr=1e-4) ##for test of pretrained
EPOCH = 150
loss_list=[]
los_min=10**10
val_loss_list=[]
ac_list=[]

for epoch in tqdm.tqdm(range(EPOCH)):
    running_loss = 0.0
    count=0
    for _, (inputs, labels) in enumerate(train_dataloader, 0):

        optimizer.zero_grad()
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        outputs = Transmodel(inputs)
        loss = criterion(outputs, labels.squeeze())
        loss.backward()
        optimizer.step()
        count=count+1

        # print statistics
        running_loss += loss.item()
    loss_loss=running_loss/count
    loss_list.append(loss_loss)
    print('epoch',epoch+1,':finished')
    print('train_loss:',loss_loss)
    with torch.no_grad():
        count=0
        running_loss=0.0
        pre=list()
        lab=list()
        for _, (inputs, labels) in enumerate(test_dataloader, 0):
            inputs=inputs.to(DEVICE)
            labels=labels.to(DEVICE)
            outputs = Transmodel(inputs)
            loss =criterion(outputs, labels.squeeze())
            running_loss += loss.item()
            count+=1
            _, predicted = torch.max(F.softmax(outputs).data, 1)
            predicted=predicted.to('cpu')
            labels=labels.to('cpu')
            predicted=predicted.tolist()
            labels=labels.tolist()
            pre.append(predicted)
            lab.append(labels)
        loss_loss=running_loss/count
        val_loss_list.append(loss_loss)
        pre=sum(pre,[])
        lab=sum(lab,[])
        print('val_loss:',loss_loss)
        cl = classification_report(lab, pre,output_dict=True)
        print(cl)
        ac_list.append(cl['accuracy'])
#         if los_min>loss_loss:
#             los_min=loss_loss
#             torch.save(Transmodel.state_dict(),'Trans_8lay_state') 

#        torch.save(Transmodel.state_dict(),'PreTrain_Trans_8lay_nodrop_mulchan_eegspec_state')
#        torch.save(Transmodel.state_dict(),'PreTest_Trans_8lay_50drop_noW_eegspec_state6')

  0%|          | 0/150 [00:00<?, ?it/s]

epoch 1 :finished
train_loss: 0.4326145324153552


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  1%|          | 1/150 [10:50<26:54:31, 650.15s/it]

val_loss: 0.41967308530962566
{'0': {'precision': 0.9201257575067208, 'recall': 0.9145419138625968, 'f1-score': 0.9173253384210048, 'support': 22081}, '1': {'precision': 0.4037940379403794, 'recall': 0.18549642079053844, 'f1-score': 0.25421198549797397, 'support': 3213}, '2': {'precision': 0.8494851605087825, 'recall': 0.8828806143967769, 'f1-score': 0.8658609992128536, 'support': 31771}, '3': {'precision': 0.8826675398384632, 'recall': 0.8273145780051151, 'f1-score': 0.8540951576279242, 'support': 9775}, '4': {'precision': 0.737515318627451, 'recall': 0.8145672954910752, 'f1-score': 0.7741287132692849, 'support': 11821}, 'accuracy': 0.8461117961887085, 'macro avg': {'precision': 0.7587175628843594, 'recall': 0.7249601645092205, 'f1-score': 0.7331244388058084, 'support': 78661}, 'weighted avg': {'precision': 0.8384068841401575, 'recall': 0.8461117961887085, 'f1-score': 0.8400766504168514, 'support': 78661}}
epoch 2 :finished
train_loss: 0.416430532114617


  1%|▏         | 2/150 [21:41<26:44:38, 650.53s/it]

...

val_loss: 0.38810210470261614
{'0': {'precision': 0.9308639714298795, 'recall': 0.9207463430098275, 'f1-score': 0.9257775146851235, 'support': 22081}, '1': {'precision': 0.4498539435248296, 'recall': 0.1437908496732026, 'f1-score': 0.2179245283018868, 'support': 3213}, '2': {'precision': 0.8612888412674116, 'recall': 0.8855245349532592, 'f1-score': 0.8732385622943696, 'support': 31771}, '3': {'precision': 0.8673216132368149, 'recall': 0.858005115089514, 'f1-score': 0.8626382103368475, 'support': 9775}, '4': {'precision': 0.7457274483578541, 'recall': 0.8489975467388546, 'f1-score': 0.7940187507417223, 'support': 11821}, 'accuracy': 0.8562057436340753, 'macro avg': {'precision': 0.7710111635633579, 'recall': 0.7314128778929316, 'f1-score': 0.7347195132719899, 'support': 78661}, 'weighted avg': {'precision': 0.847397169957456, 'recall': 0.8562057436340753, 'f1-score': 0.8479974998081792, 'support': 78661}}
epoch 33 :finished
train_loss: 0.3780446911732126


 22%|██▏       | 33/150 [5:58:01<21:09:17, 650.92s/it]

val_loss: 0.38998502015582914
{'0': {'precision': 0.915616511555417, 'recall': 0.9312078257325302, 'f1-score': 0.9233463559207867, 'support': 22081}, '1': {'precision': 0.41139240506329117, 'recall': 0.16184251478369124, 'f1-score': 0.23229841411659594, 'support': 3213}, '2': {'precision': 0.8608337685478172, 'recall': 0.8819678322998962, 'f1-score': 0.8712726594322315, 'support': 31771}, '3': {'precision': 0.8482582168864397, 'recall': 0.881841432225064, 'f1-score': 0.8647238802227015, 'support': 9775}, '4': {'precision': 0.7808947411466427, 'recall': 0.8077150833262837, 'f1-score': 0.7940785096473719, 'support': 11821}, 'accuracy': 0.8552014340016018, 'macro avg': {'precision': 0.7633991286399215, 'recall': 0.732914937673493, 'f1-score': 0.7371439638679375, 'support': 78661}, 'weighted avg': {'precision': 0.8442781359862656, 'recall': 0.8552014340016018, 'f1-score': 0.847376302668202, 'support': 78661}}
epoch 34 :finished
train_loss: 0.3779049778164856


 23%|██▎       | 34/150 [6:08:52<20:58:33, 650.98s/it]

val_loss: 0.39049588592071843
{'0': {'precision': 0.9241170352281464, 'recall': 0.9254562746252434, 'f1-score': 0.9247861700683351, 'support': 22081}, '1': {'precision': 0.3949671772428884, 'recall': 0.22471210706504824, 'f1-score': 0.28645110097202936, 'support': 3213}, '2': {'precision': 0.8443250858501364, 'recall': 0.9054483648610367, 'f1-score': 0.8738191427963913, 'support': 31771}, '3': {'precision': 0.8823340814363578, 'recall': 0.8446035805626598, 'f1-score': 0.8630566590006271, 'support': 9775}, '4': {'precision': 0.806234502302515, 'recall': 0.7701548092377971, 'f1-score': 0.7877817678362826, 'support': 11821}, 'accuracy': 0.8553667001436545, 'macro avg': {'precision': 0.7703955764120087, 'recall': 0.7340750272703571, 'f1-score': 0.7471789681347332, 'support': 78661}, 'weighted avg': {'precision': 0.8473681213772671, 'recall': 0.8553667001436545, 'f1-score': 0.8498674808839198, 'support': 78661}}


In [32]:
EPOCH = 2
loss_list=[]
los_min=10**10
val_loss_list=[]
ac_list=[]

for epoch in tqdm.tqdm(range(EPOCH)):
    with torch.no_grad():
        count=0
        running_loss=0.0
        pre=list()
        lab=list()
        for _, (inputs, labels) in enumerate(test_dataloader, 0):
            inputs=inputs.to(DEVICE)
            labels=labels.to(DEVICE)
            outputs = Transmodel(inputs)
            loss =criterion(outputs, labels.squeeze())
            running_loss += loss.item()
            count+=1
            _, predicted = torch.max(F.softmax(outputs).data, 1)
            predicted=predicted.to('cpu')
            labels=labels.to('cpu')
            predicted=predicted.tolist()
            labels=labels.tolist()
            pre.append(predicted)
            lab.append(labels)
        loss_loss=running_loss/count
        val_loss_list.append(loss_loss)
        pre=sum(pre,[])
        lab=sum(lab,[])
        print('val_loss:',loss_loss)
        cl = classification_report(lab, pre,output_dict=True)
        print(cl)
        ac_list.append(cl['accuracy'])


  0%|          | 0/2 [00:00<?, ?it/s]/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


val_loss: 0.2842285505784895



 50%|█████     | 1/2 [02:15<02:15, 135.72s/it]

{'0': {'precision': 0.9643047360451151, 'recall': 0.9388946225844336, 'f1-score': 0.9514300507724053, 'support': 196693}, '1': {'precision': 0.603467178292449, 'recall': 0.33340904788945674, 'f1-score': 0.4295154722134166, 'support': 26415}, '2': {'precision': 0.890987787309704, 'recall': 0.912200161441949, 'f1-score': 0.901469205125722, 'support': 301037}, '3': {'precision': 0.9026397564933343, 'recall': 0.875841274414967, 'f1-score': 0.8890386132046401, 'support': 97174}, '4': {'precision': 0.8108771465877725, 'recall': 0.9061449521628311, 'f1-score': 0.85586811461468, 'support': 108284}, 'accuracy': 0.8927005508475157, 'macro avg': {'precision': 0.834455320945675, 'recall': 0.7932980116987276, 'f1-score': 0.8054642911861727, 'support': 729603}, 'weighted avg': {'precision': 0.8900059403928737, 'recall': 0.8927005508475157, 'f1-score': 0.8894277139773409, 'support': 729603}}
val_loss: 0.2842285505784895



100%|██████████| 2/2 [04:33<00:00, 136.61s/it]

{'0': {'precision': 0.9643047360451151, 'recall': 0.9388946225844336, 'f1-score': 0.9514300507724053, 'support': 196693}, '1': {'precision': 0.603467178292449, 'recall': 0.33340904788945674, 'f1-score': 0.4295154722134166, 'support': 26415}, '2': {'precision': 0.890987787309704, 'recall': 0.912200161441949, 'f1-score': 0.901469205125722, 'support': 301037}, '3': {'precision': 0.9026397564933343, 'recall': 0.875841274414967, 'f1-score': 0.8890386132046401, 'support': 97174}, '4': {'precision': 0.8108771465877725, 'recall': 0.9061449521628311, 'f1-score': 0.85586811461468, 'support': 108284}, 'accuracy': 0.8927005508475157, 'macro avg': {'precision': 0.834455320945675, 'recall': 0.7932980116987276, 'f1-score': 0.8054642911861727, 'support': 729603}, 'weighted avg': {'precision': 0.8900059403928737, 'recall': 0.8927005508475157, 'f1-score': 0.8894277139773409, 'support': 729603}}
